## Preparing and cleaning files necessary for (country-level) capital stock projection workflow

## Importing necessary modules and functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import shutil
import warnings
import zipfile
from itertools import product as lstprod
from operator import itemgetter
from pathlib import Path

import dask.dataframe as ddf
import dask.delayed as delayed
import fiona
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycountry as pyctry
import rioxarray
import xarray as xr
from dask_gateway import Gateway
from py7zr import unpack_7zarchive
from tqdm.auto import tqdm

from sliiders import country_level_ypk as ypk_fn
from sliiders import settings as sset
from sliiders import spatial

# dask gateway setup
gateway = Gateway()
cluster_name = sset.DASK_IMAGE

## Maddison Project: scale change

In [3]:
## original file format was excel spreadsheet, so we will read this as is
madd = pd.read_excel(sset.DIR_YPK_RAW / "maddison_project.xlsx")

## population is in 1000s of people; we will save it to be in millions of people
madd["pop"] = madd["pop"] / 1000  ## divide by a thousand to get things in millions

## GDPpc is currently in ones of USD; we want gdp to be in millions of USD
## one USD per million people = 1 million USD per person
madd["gdp"] = madd["gdppc"] * madd["pop"]

## indexing and exporting
madd.rename(columns={"countrycode": "ccode"}, inplace=True)
madd["gdppc_unit"] = "ones of USD (constant 2011 PPP USD)"
madd["gdp_unit"] = "millions of USD (constant 2011 PPP USD)"
madd["pop_unit"] = "millions of people"
madd.set_index(["ccode", "year"], inplace=True)
madd.to_parquet(sset.DIR_YPK_INT / "maddison_project.parquet")

## UN WPP: overall populations data

### Assign country (ISO) codes: initial try with obvious cases

In [4]:
## importing data
un_df = pd.read_csv(sset.DIR_YPK_RAW / "UN_WPP2019_TotalPopulation.csv")

## let's check whether there are any with "dependencies" considered together with sov.s
for i in set(un_df.Location):
    if "ependenc" in i:
        print(i)

China (and dependencies)
United Kingdom (and dependencies)
France (and dependencies)
Netherlands (and dependencies)
United States of America (and dependencies)
New Zealand (and dependencies)
Denmark (and dependencies)


In [5]:
## very minor clean-up for iso country codes; initial
countryname_to_iso = dict([])

for i in list(set(un_df.Location)):
    name = pyctry.countries.get(name=i)
    oname = pyctry.countries.get(official_name=i)

    if name is not None or oname is not None:
        to_use = name
        if name is None:
            to_use = oname
        countryname_to_iso[i] = to_use.alpha_3
    else:
        countryname_to_iso[i] = None

## some mandotory clean-ups required
## Will not print them as there are too many, but can be checked via print command
## print(no_isos)
no_isos = [k for k, v in countryname_to_iso.items() if v is None]

In [6]:
## after examining the no_isos list, I conduct the following (manual) clean-up
to_update = {
    "Micronesia (Fed. States of)": "FSM",
    "State of Palestine": "PSE",
    "China (and dependencies)": "CHN+D",
    "China, Macao SAR": "MAC",
    "China, Hong Kong SAR": "HKG",
    "Bolivia (Plurinational State of)": "BOL",
    "Saint Helena": "SHN",
    "Holy See": "VAT",
    "Venezuela (Bolivarian Republic of)": "VEN",
    "Iran (Islamic Republic of)": "IRN",
    "United Kingdom (and dependencies)": "GBR+D",
    "New Zealand (and dependencies)": "NZL+D",
    "Dem. People's Republic of Korea": "PRK",
    "China, Taiwan Province of China": "TWN",
    "Democratic Republic of the Congo": "COD",
    "Republic of Korea": "KOR",
    "United States Virgin Islands": "VIR",
    "Denmark (and dependencies)": "DNK+D",
    "France (and dependencies)": "FRA+D",
    "United States of America (and dependencies)": "USA+D",
    "Wallis and Futuna Islands": "WLF",
    "Channel Islands": "GGY+JEY",
    "Netherlands (and dependencies)": "NLD+D",
}

## updating the ISO codes
countryname_to_iso.update(to_update)

### Detecting cases spanning multiple regions

We do not want to account for cases like "Europe" where there are multiple countries / territories / sovereignties associated with it. Therefore, we will assign, to these multiple-region cases, the code `WIDE`.

In [7]:
## re-checking for clean-ups; again, this is too long a list to print
no_isos_2 = [k for k, v in countryname_to_iso.items() if v is None]

## the whole of no_isos_2 is "WIDE"
for i, ctry in enumerate(no_isos_2):
    countryname_to_iso[ctry] = "WIDE"

## applying the dictionary to get country codes (ISO)
un_df["ccode"] = un_df.Location.map(countryname_to_iso)

### Exporting

In [8]:
os.makedirs(sset.DIR_YPK_INT, exist_ok=True)
un_df.rename(columns={"Time": "year"}, inplace=True)
un_df.set_index(["ccode", "year"], inplace=True)
un_df.to_parquet(sset.DIR_YPK_INT / "un_population.parquet")

## UN WPP: population-by-age-group

In [9]:
## attaching country codes; first import un_pop information
by_age = pd.read_csv(sset.DIR_YPK_RAW / "UN_WPP2019_Population_by_Age.csv")

## attaching the country codes
un_df_dic = dict(zip(un_df.Location, un_df.index.get_level_values("ccode")))
by_age["ccode"] = by_age.Location.map(un_df_dic)

## double checking if any are missing country codes
print("The missing-ccode rows are:", by_age[pd.isnull(by_age.ccode)].shape[0])

## saving the ccodes as indices
by_age.set_index(["ccode"], inplace=True)

## exporting
by_age.to_parquet(sset.DIR_YPK_INT / "un_population_by_age.parquet")

The missing-ccode rows are: 0


## GEG-15

In [ ]:
# cluster setup
N_CLUSTER = 20
cluster = gateway.new_cluster(worker_image=cluster_name, profile="micro")
client = cluster.get_client()
cluster.scale(N_CLUSTER)
cluster

In [11]:
@delayed
def clean_chunk(start, num, shp_path):
    with fiona.open(shp_path, "r") as shp:
        chunk = shp[start : (start + num)]
    properties = pd.DataFrame((map(itemgetter("properties"), chunk)))
    geometries = list(map(itemgetter("geometry"), chunk))
    coordinates = pd.DataFrame(
        map(itemgetter("coordinates"), geometries), columns=["lon", "lat"]
    )
    df = coordinates.merge(properties, left_index=True, right_index=True)
    return df

In [12]:
DIR_GAR = sset.DIR_GEG15_RAW / "gar-exp"
with fiona.open(DIR_GAR / "gar_exp.shp") as shp:
    num_geoms = len(shp)

data_chunked = []
for ii in range(0, num_geoms, 1000):
    data_chunked.append(clean_chunk(ii, 1000, str(DIR_GAR / "gar_exp.shp")))

In [13]:
df = ddf.from_delayed(data_chunked)

In [14]:
df = df.repartition(npartitions=16).persist()

In [15]:
os.makedirs(sset.DIR_GEG15_INT, exist_ok=True)
df.to_parquet(sset.DIR_GEG15_INT / "gar_exp.parquet")

[None]

In [16]:
cluster.scale(0)
client.close()
cluster.close()
cluster.shutdown()

## LandScan 2019

### Unzipping

In [17]:
landscan_zip = sset.DIR_LANDSCAN_RAW / f"{sset.LANDSCAN_VERS}.zip"
dir_landscan_raw = sset.DIR_LANDSCAN_RAW / sset.LANDSCAN_VERS
dir_landscan = dir_landscan_raw / f"lspop{sset.LANDSCAN_YEAR}"
path_landscan = dir_landscan / "hdr.adf"

In [19]:
if not path_landscan.exists():
    with zipfile.ZipFile(landscan_zip, "r") as zip_ref:
        zip_ref.extractall(dir_landscan_raw)

### Organizing TIF to parquet

In [ ]:
# cluster setup
N_CLUSTER = 20
cluster = gateway.new_cluster(worker_image=cluster_name, profile="micro")
client = cluster.get_client()
cluster.scale(N_CLUSTER)
cluster

#### Open raw population raster

In [21]:
pop_ds = rioxarray.open_rasterio(path_landscan, chunks={"x": 5400, "y": 10440})
pop_ds = pop_ds.squeeze().drop("band")

# Replace null values with 0's
pop_ds = pop_ds.where(pop_ds >= 0, 0)
pop_ds = pop_ds.persist()

#### Transform to dataframe

In [ ]:
pop_da = pop_ds.to_dataset(name="population")
pop_ddf = pop_da.to_dask_dataframe()
pop_ddf = pop_ddf.persist()
pop_ddf = pop_ddf[pop_ddf["population"] > 0].persist()

# Bring to local
pop_df = pop_ddf.compute()

#### Convert coordinates to indices

In [23]:
pop_df["x_ix"] = spatial.grid_val_to_ix(
    pop_df["x"].to_numpy(), sset.LANDSCAN_GRID_WIDTH
)

pop_df["y_ix"] = spatial.grid_val_to_ix(
    pop_df["y"].to_numpy(), sset.LANDSCAN_GRID_WIDTH
)

#### Drop unnecessary columns

In [24]:
pop_with_xy = pop_df.copy()
pop_df = pop_df.drop(columns=["x", "y", "spatial_ref"]).reset_index(drop=True)
pop_with_xy = pop_with_xy.drop(columns=["spatial_ref"]).reset_index(drop=True)

#### Save and shut down workers

In [26]:
os.makedirs(sset.DIR_LANDSCAN_INT, exist_ok=True)
pop_df.to_parquet(sset.DIR_LANDSCAN_INT / "population.parquet", index=False)
pop_with_xy.to_parquet(
    sset.DIR_LANDSCAN_INT / "population_with_xy.parquet", index=False
)

cluster.scale(0)
client.close()
cluster.close()
cluster.shutdown()

## CIA World Factbook: gathering GDP PPP terms

The information gathered through sources such as PWT, World Bank WDI, and OECD Regional data often lack GDP information about many of the smaller or disputed countries and territories. In order to account for these countries, we incorporate data from CIA World Factbook dataset which has not much year-to-year information but has more countries covered.

### Unzipping and organizing the files

Note that the cell directly below needs to be run **only once** since it is basically unzipping the `.7z` zip file and may take a long time to repeat over again.

In [29]:
## unzipping: this may take a long time
CIA_DIR, zip_file_name = sset.DIR_YPK_RAW, "weekly_json.7z"
shutil.register_unpack_format("7zip", [".7z"], unpack_7zarchive)
shutil.unpack_archive(CIA_DIR / zip_file_name, CIA_DIR)

In [30]:
## ordering them by time (Earlier entries first)
CIA_DIR_week = sset.DIR_YPK_RAW / "weekly_json"
file_lst = np.sort(os.listdir(CIA_DIR_week))

### Fetch necessary information from the individual `.json` files

In [32]:
def file_gdp_fetcher(filename, location=CIA_DIR_week):
    """From weekly-scraped CIA World Factbook data (in json format), gather relevant GDP
    information and save as a dictionary.

    Parameters
    ----------
    filename : str
        individual weekly-scraped CIA World Factbook data file name
    location : Path
        where the CIA World Factbook data are stored at

    overall_dict : dict
        information (in dictionary format) containing the countries' GDP information
        (in purchasing power parity) and for which year(s) those information is provided

    """

    with open(location / filename) as fp:
        data = json.load(fp)
    ctries = list(data["countries"].keys())
    ctries.sort()

    note_phrase_1 = "data are in "
    note_phrase_2 = " dollars"
    note_phrase_3 = " us dollars"

    overall_dict = dict([])
    for c in ctries:

        try:
            info = data["countries"][c]["data"]["economy"]["gdp"]
            info = info["purchasing_power_parity"]
            note = info.get("note")

            base_yr = None
            if note is not None:
                note = note.lower()
                if (note_phrase_1 in note) and (note_phrase_3 in note):
                    note_ = note.split(note_phrase_1)[1]
                    note_ = note_.split(note_phrase_3)[0]
                    base_yr = int(note_[0:4])
                elif (note_phrase_1 in note) and (note_phrase_2 in note):
                    note_ = note.split(note_phrase_1)[1]
                    note_ = note_.split(note_phrase_2)[0]
                    base_yr = int(note_[0:4])
            info_values = info.get("annual_values")
            if (info_values is not None) and (type(info_values) in [tuple, list]):
                keys = []
                values = []
                for i in info_values:
                    keys.append(int(i["date"]))
                    values.append((i["value"], int(i["date"])))
                if base_yr is not None:
                    values = [(x[0], base_yr) for x in values]
                yr_dict = dict(zip(keys, values))
                overall_dict[c] = yr_dict

        except KeyError:
            continue

    return overall_dict

In [33]:
## individual results of the file_gdp_fetcher function stored in a list
lst_results = []
for f in tqdm(file_lst):
    lst_results.append(file_gdp_fetcher(f))

  0%|          | 0/693 [00:00<?, ?it/s]

### Updating the individual dictionaries with the most recent information

In [34]:
def update_one_with_two(dict1, dict2):
    """For simple updating of dictionaries, from `dict2` onto `dict1` in order to make
    sure that all relevant CIA World Factbook data are gathered

    Parameters
    ----------
    dict1 : dict
        dictionary to implement the updates onto
    dict2 : dict
        dictionary to gather new information from

    Returns
    -------
    dict_ : dict
        updated dictionary containing the information of both dictionaries

    """

    dict_ = dict(dict1)
    lst1 = list(dict1.keys())

    for key in dict2.keys():
        if key not in lst1:
            dict_[key] = dict2[key]
            continue

        subdict = dict2[key]
        subkeys = list(subdict.keys())
        for subkey in subkeys:
            dict_[key][subkey] = subdict[subkey]

    return dict_

In [35]:
i = 0
for res in tqdm(lst_results[1:]):
    if i == 0:
        midres = update_one_with_two(lst_results[0], res)
    else:
        midres = update_one_with_two(midres, res)
    i += 1

  0%|          | 0/692 [00:00<?, ?it/s]

### Saving into a long-panel format dataframe

In [36]:
ctry_dfs = []
for i in midres.keys():
    info = midres[i]
    i_k = list(info.keys())
    i_v = [info[i_k_] for i_k_ in i_k]
    ctry_info = [[i, i_k[l]] + list(i_v[l]) for l in range(len(i_k))]
    ctry_df = pd.DataFrame(ctry_info, columns=["country", "year", "gdp", "ppp_year"])
    ctry_dfs.append(ctry_df)
ctry_agg_df = pd.concat(ctry_dfs, axis=0)
ctry_agg_df["country"] = [x.replace("_", " ") for x in ctry_agg_df["country"]]
ctry_agg_df.set_index(["country", "year"], inplace=True)
ctry_agg_df.sort_index(inplace=True)

### Assigning countrycodes

In [37]:
## let's use the UN populations data, since it should have the most countries
## to match names with values
un_loc = sset.DIR_YPK_INT
unpop = pd.read_parquet(un_loc / "un_population.parquet").reset_index()

unpop["Location_lower"] = [x.lower() for x in unpop.Location]
initial_cleanup = dict(zip(unpop.Location_lower, unpop.ccode))

## attaching the cleaned countrycodes
initial_df = [list(initial_cleanup.keys()), list(initial_cleanup.values())]
initial_df = pd.DataFrame(
    np.array(initial_df).T, columns=["country", "ccode"]
).set_index(["country"])
ctry_agg_df = ctry_agg_df.merge(
    initial_df, left_index=True, right_index=True, how="left"
)

In [38]:
## checking which didn't get country codes
cia_ccodes_only = ctry_agg_df.reset_index()[["country", "ccode"]].drop_duplicates()
unknown_case = []
for i, case in enumerate(cia_ccodes_only["ccode"]):
    if pd.isnull(case):
        unknown_case.append(cia_ccodes_only["country"].values[i])
unknown_case = np.sort(np.unique(unknown_case))
print(unknown_case)

['bahamas the' 'bolivia' 'brunei' 'burma' 'cape verde'
 'congo democratic republic of the' 'congo republic of the'
 "cote d' ivoire" 'curacao' 'czech republic' 'east timor' 'european union'
 'falkland islands islas malvinas' 'gambia the' 'gaza strip' 'guernsey'
 'guinea bissau' 'hong kong' 'iran' 'jersey' 'korea north' 'korea south'
 'kosovo' 'laos' 'macau' 'macedonia' 'micronesia federated states of'
 'moldova' 'netherlands antilles' 'russia'
 'saint helena ascension and tristan da cunha' 'saint martin'
 'sint maarten' 'swaziland' 'syria' 'taiwan' 'tanzania' 'timor leste'
 'united states' 'venezuela' 'vietnam' 'virgin islands'
 'wallis and futuna' 'west bank']


In [39]:
## manual cleanup
unknown_case_ccodes = ["BHS", "BOL", "BRN", "MMR", "CPV", "COD", "COG", "CIV", "CUW"]
unknown_case_ccodes += ["CZE", "TLS", "-", "FLK", "GMB", "-", "GGY", "GNB", "HKG"]
unknown_case_ccodes += ["IRN", "JEY", "PRK", "KOR", "KO-", "LAO", "MAC", "MKD", "FSM"]
unknown_case_ccodes += ["MDA", "-", "RUS", "SHN", "MAF", "SXM", "SWZ", "SYR", "TWN"]
unknown_case_ccodes += ["TZA", "TLS", "USA", "VEN", "VNM", "VIR", "WLF", "-"]

## double-checking the names' lengths
print(len(unknown_case) == len(unknown_case_ccodes))

## getting a dataframe
update_df = pd.DataFrame(data={"country": unknown_case, "ccode2": unknown_case_ccodes})
update_df.set_index(["country"], inplace=True)
ctry_agg_df = ctry_agg_df.merge(
    update_df, left_index=True, right_index=True, how="left"
)
ctry_agg_df.loc[pd.isnull(ctry_agg_df.ccode), "ccode"] = ctry_agg_df.loc[
    pd.isnull(ctry_agg_df.ccode), "ccode2"
].values

True


### Fetching the PPP conversion rates (to constant 2017 PPP USD), and applying the conversion rates

Also, turn it into millions of USD (currently in ones of USD)

In [40]:
ppp_to_17 = ypk_fn.ppp_conversion_specific_year(2017, to=True, extrap_sim=True)

Fetching information from PWT...


  0%|          | 0/128 [00:00<?, ?it/s]

Filling in the missing countries...
...done


In [41]:
## neutral assumption when conversion rates are missing
ctry_agg_df = (
    ctry_agg_df.reset_index()
    .set_index(["ccode", "year"])
    .drop(["ccode2"], axis=1)
    .merge(ppp_to_17, left_index=True, right_index=True, how="left")
)
ctry_agg_df.loc[pd.isnull(ctry_agg_df.conv), "conv"] = 1

## first, divide by 1000000
ctry_agg_df["gdp"] = ctry_agg_df["gdp"] / 1000000

## applying the conversion by multiplying
ctry_agg_df["gdp_ppp2017_currUSD"] = ctry_agg_df["gdp"] * ctry_agg_df["conv"]

### Attaching the US deflators and generating constant 2017 PPP USD values

Note that while they are now in PPP of 2017, they are yet to be turned into constant 2017 PPP (since they are in current USD, for many). Therefore, we will need to fetch the US deflators (using `pl_gdpo` from PWT).

In [42]:
pwt = pd.read_excel(sset.PATH_PWT_RAW).rename(columns={"countrycode": "ccode"})
pwt.set_index(["ccode", "year"], inplace=True)

us_defla = (
    pwt.loc["USA", ["pl_gdpo"]]
    .reset_index()
    .rename(columns={"pl_gdpo": "pl_usa", "year": "ppp_year"})
)
ctry_agg_df = (
    ctry_agg_df.reset_index()
    .merge(us_defla, on=["ppp_year"], how="left")
    .set_index(["ccode", "year"])
)

## generating constant 2017 ppp
ctry_agg_df["gdp_constant2017ppp"] = (
    ctry_agg_df["gdp_ppp2017_currUSD"] / ctry_agg_df["pl_usa"]
)

ctry_agg_df_reorg = ctry_agg_df[["gdp_constant2017ppp", "country"]].sort_index()

### Checking for redundancies in country (ISO) codes

Except when there aren't any country-codes attached, these redundancies are occurring because there have been changes to the countries' names over the years or there are multiple names for one country. We will use the following rule to remove some of the overlaps:
- SHN: Take only `saint helena ascension and tristan da cunha`
- CZE: For 2006-2012, use `czech republic` information; for 2013 and onwards, use `czechia` information.
- MKD: For 2006-2014, use `macedonia` information; for 2015 and onwards, use `north macedonia` information.
- SWZ: For 2006-2014, use `swaziland` information; for 2015 and onwards, use `eswatini` information.
- CPV: For 2006-2011, use `cape verde` information; for 2012 and onwards, use `cabo verde` information.
- TLS: Take only `timor leste`.

In [46]:
reorg_ccodes = ctry_agg_df_reorg.reset_index()[["ccode", "country"]].drop_duplicates()
reorg_ccodes.set_index(["ccode"], inplace=True)
for i, ccode in enumerate(np.unique(reorg_ccodes.index.values)):
    countrycases = reorg_ccodes.loc[ccode, "country"]
    if (ccode != "-") and (type(countrycases) != str):
        print(ccode, countrycases.values)

CPV ['cape verde' 'cabo verde']
CZE ['czech republic' 'czechia']
MKD ['macedonia' 'north macedonia']
SHN ['saint helena' 'saint helena ascension and tristan da cunha']
SWZ ['swaziland' 'eswatini']
TLS ['east timor' 'timor leste']


In [47]:
redundant_ones = ["SHN", "CZE", "MKD", "SWZ", "CPV", "TLS"]
ctry_agg_df_redun = ctry_agg_df_reorg.reset_index()

ctry_shn = ctry_agg_df_redun.loc[
    ctry_agg_df_redun.country == "saint helena ascension and tristan da cunha"
].set_index(["ccode", "year"])

ctry_cze = ctry_agg_df_redun.loc[
    ((ctry_agg_df_redun.country == "czechia") & (ctry_agg_df_redun.year >= 2013))
    | (
        (ctry_agg_df_redun.country == "czech republic")
        & (ctry_agg_df_redun.year <= 2012)
    )
].set_index(["ccode", "year"])

ctry_mkd = ctry_agg_df_redun[
    ((ctry_agg_df_redun.country == "macedonia") & (ctry_agg_df_redun.year <= 2014))
    | (
        (ctry_agg_df_redun.country == "north macedonia")
        & (ctry_agg_df_redun.year >= 2015)
    )
].set_index(["ccode", "year"])

ctry_swz = ctry_agg_df_redun[
    ((ctry_agg_df_redun.country == "swaziland") & (ctry_agg_df_redun.year <= 2014))
    | ((ctry_agg_df_redun.country == "eswatini") & (ctry_agg_df_redun.year >= 2015))
].set_index(["ccode", "year"])

ctry_cpv = ctry_agg_df_redun[
    ((ctry_agg_df_redun.country == "cape verde") & (ctry_agg_df_redun.year <= 2011))
    | ((ctry_agg_df_redun.country == "cabo verde") & (ctry_agg_df_redun.year >= 2012))
].set_index(["ccode", "year"])

ctry_tls = ctry_agg_df_redun.loc[
    ctry_agg_df_redun.country == "timor leste", :
].set_index(["ccode", "year"])

In [48]:
ctry_agg_df_final = ctry_agg_df_reorg[
    ~ctry_agg_df_reorg.index.get_level_values("ccode").isin(
        ["-", "WIDE"] + redundant_ones
    )
].copy()

ctry_agg_df_final = pd.concat(
    [ctry_agg_df_final, ctry_shn, ctry_cze, ctry_mkd, ctry_swz, ctry_cpv, ctry_tls],
    axis=0,
).sort_index()

### Adding those that are not in the files

**Tokelau `TKL`**

According to Tokelau government (link [here](https://www.tokelau.org.nz/Bulletin/April+2017/GDP+first.html)), its PPP USD was 10 million (in 2017). So we will fill this in.

In [49]:
tkl = pd.DataFrame(
    [
        ["TKL", 2017, 10, "tokelau"],
    ],
    columns=["ccode", "year", "gdp_constant2017ppp", "country"],
).set_index(["ccode", "year"])
ctry_agg_df_final = pd.concat([ctry_agg_df_final, tkl], axis=0)

**Saint Helena (`SHN`)**

I update the latest values using the CIA World Factbook's January 7, 2021 vintage (link [here](https://www.cia.gov/the-world-factbook/)). For `SHN`, it is said that the 2009 value of GDP (in constant 2009 PPP USD) is 31.1 million, but we do not have the explicit PPP conversion for `SHN`. Since `SHN` is a British territory, `GBR` PPP rates are used.

In [50]:
shn_rate = ppp_to_17.loc[("SHN", 2009), "conv"]
us_def09 = pwt.loc[("USA", 2009), "pl_gdpo"]
shn = pd.DataFrame(
    [
        ["SHN", 2009, shn_rate / us_def09 * 31.1, "saint helena"],
    ],
    columns=["ccode", "year", "gdp_constant2017ppp", "country"],
).set_index(["ccode", "year"])

ctry_agg_df_final = pd.concat([ctry_agg_df_final, shn], axis=0)

**Vatican (`VAT`)**

While not in the latest CIA World Factbook, the 2000 version has some information about Vatican city (archived [here](https://www.encyclopedia.com/places/spain-portugal-italy-greece-and-balkans/italian-political-geography/vatican-city)) which we will be able to use. It says that the 1999 estimate of the Vatican GDP (assuming it's constant 1999 PPP) was 21 million USD. Let us use the PPP conversion rates of Italy.

In [51]:
vat_rate = ppp_to_17.loc[("VAT", 1999), "conv"]
us_def99 = pwt.loc[("USA", 1999), "pl_gdpo"]
vat = pd.DataFrame(
    [
        ["VAT", 1999, vat_rate / us_def99 * 21, "vatican"],
    ],
    columns=["ccode", "year", "gdp_constant2017ppp", "country"],
).set_index(["ccode", "year"])

ctry_agg_df_final = pd.concat([ctry_agg_df_final, vat], axis=0)

### Exporting

In [53]:
ctry_agg_df_final.sort_index(inplace=True)
ctry_agg_df_final.rename(columns={"gdp_constant2017ppp": "cia_rgdpna"}, inplace=True)
ctry_agg_df_final.to_parquet(
    sset.DIR_YPK_INT / "cia_wf_gdp_constant_2017_ppp_usd_ver.parquet"
)